# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
df = pd.read_csv(url, index_col=0)
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
df.shape

(6000, 61)

In [3]:
X = df.drop(['url', 'Popular'], axis=1)
y = df['Popular']

In [4]:
y.mean()

0.5

In [5]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

dt=DecisionTreeClassifier().fit(X_train,y_train)
y1_pred=dt.predict(X_test)
lm=LogisticRegression(solver='liblinear',C=1e9).fit(X_train,y_train)
y2_pred=lm.predict(X_test)

In [7]:
print('Desition Tree Accuracy:',round(metrics.accuracy_score(y_test,y1_pred),2),
'\nLogistic Regression Accuracy:',round(metrics.accuracy_score(y_test,y2_pred),2))

Desition Tree Accuracy: 0.55 
Logistic Regression Accuracy: 0.63


In [8]:
print('Desition Tree F1 score:',round(metrics.f1_score(y_test,y1_pred),2),
'\nLogistic Regression F1 score:',round(metrics.f1_score(y_test,y2_pred),2))

Desition Tree F1 score: 0.54 
Logistic Regression F1 score: 0.61


# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [10]:
import numpy as np

n_estimators = 300
# set a seed for reproducibility
np.random.seed(123)

n_samples = X_train.shape[0]

# create bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(n_estimators)]

In [11]:
seeds = np.random.randint(1, 10000, size=n_estimators)

models = {}
for i in range(n_estimators):
    if i<101:
        models[i] = DecisionTreeClassifier(max_depth=None, random_state=123)
        models[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])
    elif i<201:
        models[i] = DecisionTreeClassifier(max_depth=2, random_state=123)
        models[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])
    else:
        models[i] = LogisticRegression(solver='liblinear',C=1e9, random_state=123)
        models[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [12]:
y_pred_df = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_df.iloc[:, i] = models[i].predict(X_test)

y_pred_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1483,1,1,0,0,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2185,1,1,1,1,0,1,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2520,1,0,0,0,1,1,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1
3721,1,0,1,0,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
3727,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
y_pred = (y_pred_df.sum(axis=1) >= (n_estimators / 2)).astype(np.int)

print('Ensamble F1 score:',round(metrics.f1_score(y_pred, y_test),4),
'\nEnsamble accuracy score:',round(metrics.accuracy_score(y_pred, y_test),4))

Ensamble F1 score: 0.6246 
Ensamble accuracy score: 0.6353


# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [14]:
print('Probabilidad de predicción de 1 en los modelos:',
      round(y_pred_df.sum(axis=0).sum()/(y_pred_df.shape[0]*y_pred_df.shape[1]),3))

Probabilidad de predicción de 1 en los modelos: 0.481


In [15]:
f1=0
for prob in range(0,101):
    y_pred=(y_pred_df.sum(axis=1)/y_pred_df.shape[1]>=prob/100).astype(np.int)
    if metrics.f1_score(y_pred, y_test)>f1:
        f1=metrics.f1_score(y_pred, y_test)
        proba=prob/100
print('f1:',round(f1,3),'\nProba:',round(proba,3))

f1: 0.711 
Proba: 0.14


# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [16]:
samples_oob = []
for sample in samples:
    samples_oob.append(sorted(set(range(n_samples)) - set(sample)))

In [17]:
errors = np.zeros(n_estimators)

for i in range(n_estimators):
    y_pred_ = models[i].predict(X_train.iloc[samples_oob[i]])
    errors[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_)
    
alpha = (1 - errors) / (1 - errors).sum()
weighted_sum_1 = ((y_pred_df) * alpha).sum(axis=1)

In [18]:
y_pred = (weighted_sum_1 >= 0.5).astype(np.int)

print('Ensamble F1 score:',round(metrics.f1_score(y_pred, y_test),4),
'\nEnsamble accuracy score:',round(metrics.accuracy_score(y_pred, y_test),4))

Ensamble F1 score: 0.6249 
Ensamble accuracy score: 0.6367


# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [19]:
print('Probabilidad de predicción de 1 en los modelos:',
      round(weighted_sum_1.sum()/(y_pred_df.shape[0]),3))

Probabilidad de predicción de 1 en los modelos: 0.48


In [20]:
f1=0
for prob in range(0,11):
    y_pred=(weighted_sum_1>=prob/10).astype(np.int)
    if metrics.f1_score(y_pred, y_test)>f1:
        f1=metrics.f1_score(y_pred, y_test)
        proba=prob/100
print('f1:',round(f1,3),'\nProba:',round(proba,3))

f1: 0.704 
Proba: 0.01


# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [21]:
X_train_2 = pd.DataFrame(index=X_train.index, columns=list(range(n_estimators)))

for i in range(n_estimators):
    X_train_2[i] = models[i].predict(X_train)
X_train_2.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
2420,1,1,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1225,1,0,1,0,1,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
324,0,0,0,0,0,0,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1
5863,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,0,1
86,0,1,0,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1


In [22]:
from sklearn.linear_model import LogisticRegressionCV

lr = LogisticRegressionCV(cv = 5 )
lr.fit(X_train_2, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [23]:
y_pred = lr.predict(y_pred_df)

print('Ensamble F1 score:',round(metrics.f1_score(y_pred, y_test),4),
'\nEnsamble accuracy score:',round(metrics.accuracy_score(y_pred, y_test),4))

Ensamble F1 score: 0.65 
Ensamble accuracy score: 0.646


In [38]:
f1=0
for prob in range(0,101):
    y_pred=(lr.predict_proba(y_pred_df)[:,1]>=prob/100).astype(np.int)
    if metrics.f1_score(y_pred, y_test)>f1:
        f1=metrics.f1_score(y_pred, y_test)
        proba=prob/100
print('f1:',round(f1,3),'\nProba:',round(proba,3))

f1: 0.712 
Proba: 0.32
